In [1]:
import kagglehub

# Download latest version

data_path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

In [2]:
# For loading data and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_data = pd.read_csv(data_path + "/" + "train.csv", index_col=0)
test_data = pd.read_csv(data_path + "/" + "test.csv")

In [3]:
train_data.head()

,file_name,label
0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1


In [4]:
# Creates a specific to each image
train_full_path = train_data["file_name"].apply(lambda image_path: data_path + "/" + image_path).values
test_full_path = test_data["id"].apply(lambda image_path: data_path + "/" + image_path).values

In [5]:
import tensorflow as tf